In [1]:
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle
import numpy as np
import os

In [2]:
file  = open('data.txt', 'r', encoding = "utf8")

In [3]:
lines=[]
for i in file:
    lines.append(i)

In [4]:
data=""
for i in lines:
    data = ' '.join(lines)

In [5]:
 data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')

In [6]:
data = data.split()
data = ' '.join(data)

In [7]:
data[:1000]

"THE ADVENTURES OF SHERLOCK HOLMES Arthur Conan Doyle Table of contents A Scandal in Bohemia The Red-Headed League A Case of Identity The Boscombe Valley Mystery The Five Orange Pips The Man with the Twisted Lip The Adventure of the Blue Carbuncle The Adventure of the Speckled Band The Adventure of the Engineer's Thumb The Adventure of the Noble Bachelor The Adventure of the Beryl Coronet The Adventure of the Copper Beeches A SCANDAL IN BOHEMIA Table of contents Chapter 1 Chapter 2 Chapter 3 CHAPTER I To Sherlock Holmes she is always the woman. I have seldom heard him mention her under any other name. In his eyes she eclipses and predominates the whole of her sex. It was not that he felt any emotion akin to love for Irene Adler. All emotions, and that one particularly, were abhorrent to his cold, precise but admirably balanced mind. He was, I take it, the most perfect reasoning and observing machine that the world has seen, but as a lover he would have placed himself in a false positio

In [8]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

In [9]:
pickle.dump(tokenizer, open('token.pkl', 'wb'))

In [10]:
sequence_data = tokenizer.texts_to_sequences([data])[0]

In [11]:
sequence_data[:10]

[1, 1561, 5, 129, 34, 647, 4498, 4499, 226, 5]

In [12]:
len(sequence_data)

105879

In [13]:
vocab_size = len(tokenizer.word_index)+1

In [14]:
vocab_size

8200

In [15]:
sequence=[]
for i in range(3, len(sequence_data)):
    words = sequence_data[i-3:i+1]
    sequence.append(words)

In [16]:
len(sequence)

105876

In [17]:
sequence = np.array(sequence)

In [18]:
sequence

array([[   1, 1561,    5,  129],
       [1561,    5,  129,   34],
       [   5,  129,   34,  647],
       ...,
       [  28,    1, 8198, 8199],
       [   1, 8198, 8199, 3187],
       [8198, 8199, 3187, 3186]])

In [19]:
X = []
y = []

# Assuming `sequence` is defined elsewhere
for i in sequence:
    X.append(i[0:3])
    y.append(i[3])

X = np.array(X)
y = np.array(y)



In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(vocab_size, 50))  # Removed input_length argument
model.add(LSTM(256, return_sequences=True))  # LSTM with return_sequences
model.add(LSTM(256))  # LSTM without return_sequences
model.add(Dense(256, activation='relu'))  # Fully connected layer
model.add(Dense(vocab_size, activation="softmax"))  # Output layer

In [23]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding (Embedding)                │ (None, 3, 50)               │         410,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ (None, 3, 256)              │         314,368 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 8200)                │       2,107,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 10,268,618 (39.17 MB)

 Trainable params: 3,422,872 (13.06 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 6,845,746 (26.11 MB)

In [22]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

checkpoint = ModelCheckpoint("next_words.keras", monitor="loss", verbose=1, save_best_only=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=Adam(learning_rate=0.001))

model.fit(X, y, epochs=2, batch_size=32, callbacks=[checkpoint])  # Adjusted batch size


Epoch 1/2
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - loss: 6.6417
Epoch 1: loss improved from inf to 6.35234, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 120s 35ms/step - loss: 6.6416
Epoch 2/2
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - loss: 5.7884
Epoch 2: loss improved from 6.35234 to 5.74378, saving model to next_words.keras
3309/3309 ━━━━━━━━━━━━━━━━━━━━ 122s 37ms/step - loss: 5.7884
